In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data.data import Data
data = Data()

In [ ]:
from src.data.api import ScApiDevice

outdoor_kitsdf = ScApiDevice.get_world_map(min_date='2020-03-01', max_date='2020-04-01', tags = ['outdoor'], tag_method = 'all', full = True)
outdoor_kits = list(outdoor_kitsdf.index)
print ('Outdoor Kits')
print (outdoor_kits)
print (len(outdoor_kits))

In [ ]:
from src.visualization.visualization_tools import device_history_map
import pandas as pd
options = {'fillOpacity':1.0, 'radius': 5000, 'zoom': 3.5}
device_history_map(map_type = 'static', dataframe = outdoor_kitsdf, options = options)

In [ ]:
from src.saf import config
from src.visualization.visualization_tools import *
config.cached_data_margin = 6

In [ ]:
blueprint = 'sck_21'
_sck_testname_outdoor = 'COVID_SCK_WORLD_OUTDOOR'

# Here we specify the different options for data load
options = {'frequency': '60Min',
            'min_date': None,
            'max_date': None,
            'clean_na': None
            }

# Add devices to list
from src.data.device import Device

tdevices = list()
for device in outdoor_kits:
    tdevices.append(Device(blueprint = blueprint, descriptor = {'id': str(device), 
                                                                'source': 'api'
                                                                }))
# We load the data based on the input above
sck_testname_outdoor = data.load_devices(_sck_testname_outdoor, tdevices, options = options)

In [ ]:
sck_testname_outdoor='2020-04_INT_COVID_SCK_WORLD_OUTDOOR'
data.load_test(sck_testname_outdoor, options = options)

In [ ]:
for metric in ['NOISE_A']:
    for device in data.tests[sck_testname_outdoor].devices:
        data.tests[sck_testname_outdoor].devices[str(device)].add_metric({f'{metric}_AVG_7D': {'process': 'rolling_avg',
                                                                'kwargs': {'name': metric,
                                                                           'window_size': '7D'}
                                                               }
                                                     })

In [ ]:
data.process_test('2020-04_INT_COVID_SCK_WORLD_OUTDOOR')

In [ ]:
m = device_metric_map(data.tests['2020-04_INT_COVID_SCK_WORLD_OUTDOOR'], channel = "NOISE_A_AVG_3D",\
                  start_date = '2020-02-01', end_date = '2020-04-17',\
                  options = {'period': '1D'})

#m.save('/Users/macoscar/Desktop/Noise_map.html')

In [ ]:
m.save('/Users/macoscar/Desktop/Noise_map.html')

In [ ]:
m = device_metric_map(data.tests['2020-04_INT_COVID_SCK_WORLD_OUTDOOR'], channel = "PM_1",\
                  start_date = '2020-01-01', end_date = '2020-04-17',\
                  options = {'period': '1D'})
m.save('/Users/macoscar/Desktop/PM_map.html')

In [ ]:
from src.visualization.visualization import Plot

In [ ]:
plot_description = {"type": "timeseries",
                    "library": "matplotlib",
                    "data": {"test": sck_testname_outdoor,
                                    "traces": {"1": {"device": [9946],
                                                    "channel" : "NOISE_A",
                                                    "subplot": 1},
                                               "2": {"device": [9946],
                                                    "channel" : "NOISE_A_AVG_3D",
                                                    "subplot": 1}, 
                                              "3": {"device": [10751],
                                                    "channel" : "NOISE_A",
                                                    "subplot": 2},
                                              "4": {"device": [10751],
                                                    "channel" : "NOISE_A_AVG_3D",
                                                    "subplot": 2},
                                              "5": {"device": [9575],
                                                    "channel" : "NOISE_A",
                                                    "subplot": 3},
                                              "6": {"device": [9575],
                                                    "channel" : "NOISE_A_AVG_3D",
                                                    "subplot": 3}}},
                    "options": {"show_plot": True,
                                "export_path": '/Users/macoscar/Desktop/IAAC-Article', 
                                "file_name": 'Noise-world',
                                "frequency": '2H',
                                "clean_na": None,
                                "max_date": None,
                                "min_date": '2020-02-01'},
                    "formatting": {"xlabel": "Date",
                                   "ylabel": {1: "Noise (dBA Scale)", 2: "Noise (dBA Scale)", 3: "Noise (dBA Scale)"},
                                   "yrange": {1: [45, 75], 2: [45, 75], 3: [45, 75]},
                                   "xrange": {1: ['2020-01-29', '2020-04-25'], 2: ['2020-01-29', '2020-04-25'], 3: ['2020-01-29', '2020-04-25']},
                                   "title": "Noise Evolution - During COVID-19 lockdown",
                                   "sharex":True,
                                   "grid": True,
                                   "legend": False,
                                   "fontsize": 13,
                                   "height": 10,
                                   "width": 20,
                                   "suptitle_factor": 0.92,
                                   "palette": "gray_r",
                                   "style": "seaborn-whitegrid",
                                   "decorators": {'axvline': ['2020-03-15', '2020-04-16'],
                                                  'xtext': {'2020-03-16': 'Lockdown starts'},
                                                  }
                                  }
                    }

plot = Plot(plot_description)
plot.plot(data)
plot.export()